In [ ]:
#!pip uninstall tensorflow
!pip install torch torchvision -U

In [1]:
!git clone https://github.com/Alberknyis/bad-teaching-unlearning.git

Cloning into 'bad-teaching-unlearning'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 30 (delta 12), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (30/30), 33.75 KiB | 33.75 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [1]:
import sys
sys.path.insert(0,'/content/bad-teaching-unlearning')

In [2]:
import torch

from dataset import *
from model import ResNet18
from unlearn import *
from metrics import UnLearningScore
from utils import *
from torch.utils.data import DataLoader

In [3]:
#import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()

In [4]:
train_ds = CustomCIFAR100(root='.', train=True,download=True, transform=transform_train)
valid_ds = CustomCIFAR100(root='.', train=False,download=True, transform=transform_train)

batch_size = 256
#batch_size = 128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=32, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=32, pin_memory=True)

#train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=8, pin_memory=True)
#valid_dl = DataLoader(valid_ds, batch_size, num_workers=8, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
num_classes = 100
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[label].append((img, label, clabel))

classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[label].append((img, label, clabel))

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
# train the model
#device = 'cuda'
model = ResNet18(num_classes = 20, pretrained = True).to(device)
#epochs = 5
epochs = 1
history = fit_one_cycle(epochs, model, train_dl, valid_dl, device = device)
torch.save(model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [0], last_lr: 0.00100, train_loss: 1.2617, val_loss: 1.0929, val_acc: 66.8262


In [8]:
torch.save(model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt")

In [7]:
# load the trained model
#device = 'cuda'
model = ResNet18(num_classes = 20, pretrained = True).to(device)
model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location='cuda'))

<All keys matched successfully>

# Forgetting Rocket
The Rocket is class 69 in CIFAR100 and belongs to Super Class 19 (Vehicles) in CIFAR Super 20.

In [8]:
# Getting the forget and retain validation data
forget_valid = []
forget_classes = [69]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))

forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)
import random
retain_train_subset = random.sample(retain_train, int(0.3*len(retain_train)))
retain_train_subset_dl = DataLoader(retain_train_subset, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [9]:
# Performance of Fully trained model on retain set
evaluate(model, retain_valid_dl, device)

{'Loss': 1.0884790420532227, 'Acc': 66.85185241699219}

In [10]:
# Performance of Fully trained model on retain set
evaluate(model, forget_valid_dl, device)

{'Loss': 0.4739783704280853, 'Acc': 82.0}

## Retrain the model from Scratch
Create Retrained Model (Gold model). This is the model trained from scratch without forget data.

In [11]:
#device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
epochs = 5
epochs = 1
history = fit_one_cycle(epochs, gold_model, retain_train_dl, retain_valid_dl, device = device)
torch.save(gold_model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt")

Epoch [0], last_lr: 0.00100, train_loss: 1.2404, val_loss: 1.0045, val_acc: 68.1234


In [12]:
#device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
gold_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt", map_location=device))

<All keys matched successfully>

In [13]:
# evaluate gold model on forget set
evaluate(gold_model, forget_valid_dl, device)

{'Loss': 6.002573490142822, 'Acc': 2.0}

In [14]:
# evaluate gold model on retain set
evaluate(gold_model, retain_valid_dl, device)

{'Loss': 1.0045000314712524, 'Acc': 68.1234130859375}

## UnLearning via proposed method

In [15]:
#device = 'cuda'
unlearning_teacher = ResNet18(num_classes = 20, pretrained = False).to(device).eval()
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))
model = model.eval()

KL_temperature = 1

optimizer = torch.optim.Adam(student_model.parameters(), lr = 0.0001)

blindspot_unlearner(model = student_model, unlearning_teacher = unlearning_teacher, full_trained_teacher = model,
          retain_data = retain_train_subset, forget_data = forget_train, epochs = 1, optimizer = optimizer, lr = 0.0001,
          batch_size = 256, num_workers = 32, device = device, KL_temperature = KL_temperature)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2949: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: aten::kl_div: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differen

Epoch 1 Unlearning Loss 0.010300659574568272


In [16]:
# performance of unlearned model on forget set
evaluate(student_model, forget_valid_dl, device)

{'Loss': 2.4537575244903564, 'Acc': 13.0}

In [17]:
# performance of unlearned model on retain set
evaluate(student_model, retain_valid_dl, device)

{'Loss': 1.0236799716949463, 'Acc': 67.97853088378906}

### Measure ZRF (Unlearning Score)

In [18]:
#print("Initial Score: {}".format(UnLearningScore(model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("Our Score: {}".format(UnLearningScore(student_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("Gold Score: {}".format(UnLearningScore(gold_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, 'cuda')))
print("Initial Score: {}".format(UnLearningScore(model, unlearning_teacher, forget_valid_dl, 256, device)))
print("Our Score: {}".format(UnLearningScore(student_model, unlearning_teacher, forget_valid_dl, 256, device)))
print("Gold Score: {}".format(UnLearningScore(gold_model, unlearning_teacher, forget_valid_dl, 256, device)))
print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, device)))

Initial Score: 0.860691249370575
Our Score: 0.9791713356971741
Gold Score: 0.9561689496040344
JS Div: 0.024255990982055664


## Unlearning using Amnesiac unlearning

In [19]:
unlearninglabels = list(range(20))
unlearninglabels.remove(19)
unlearning_train_set = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, random.choice(unlearninglabels)))



for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, clabel))

In [20]:
unlearning_train_set_dl = DataLoader(unlearning_train_set, batch_size, num_workers = 32, pin_memory = True, shuffle = True)

In [21]:
#device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = True).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = 'cuda'))
epochs = 3

history = fit_one_unlearning_cycle(epochs, student_model, unlearning_train_set_dl, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.5242, val_loss: 0.5378, val_acc: 83.0200
Epoch [1], last_lr: 0.00010, train_loss: 0.3309, val_loss: 0.5265, val_acc: 84.0671
Epoch [2], last_lr: 0.00010, train_loss: 0.2036, val_loss: 0.5543, val_acc: 84.3785


In [22]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

Forget Performance: {'Loss': 3.6175918579101562, 'Acc': 4.0}
Retain Performance: {'Loss': 0.5542724132537842, 'Acc': 84.37849426269531}


## Unlearning using UNSIR (Class 0)

In [23]:
num_classes = 20
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[clabel].append((img, label, clabel))

classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[clabel].append((img, label, clabel))

In [24]:
# Getting the forget and retain validation data
forget_valid = []
forget_classes = [0]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))

forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [25]:
#collect some samples from each class
num_samples = 500
retain_samples = []
for i in range(num_classes):
    if i not in forget_classes:
        retain_samples += classwise_train[i][:num_samples]

In [26]:
noise_batch_size = 256

In [27]:
#device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))

<All keys matched successfully>

In [28]:
noise = UNSIR_noise(batch_size, 3, 224, 224).to(device)

In [31]:
forget_class_label = 0
#num_epochs = 250
num_epochs = 1

In [32]:
noise =  UNSIR_noise_train(noise, student_model, forget_class_label, num_epochs,\
                           noise_batch_size, device=device)

Loss: 751.0631713867188


In [33]:
#takes 19 years
noisy_loader = UNSIR_create_noisy_loader(noise, forget_class_label\
                                         , retain_samples, batch_size, device=device)

In [34]:
#impair step
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, noisy_loader, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.7036, val_loss: 0.7013, val_acc: 79.1133


In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

In [ ]:
#repair step
other_samples = []
for i in range(len(retain_samples)):
    other_samples.append((retain_samples[i][0].cpu(), torch.tensor(retain_samples[i][2]),
                            torch.tensor(retain_samples[i][2])))

heal_loader = torch.utils.data.DataLoader(other_samples, batch_size=batch_size, shuffle = True)
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, heal_loader, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.0178, val_loss: 0.6395, val_acc: 84.6217


In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

Forget Performance: {'Loss': 4.577937602996826, 'Acc': 17.43404197692871}
Retain Performance: {'Loss': 0.639464259147644, 'Acc': 84.62171173095703}
